# Convert the imagery in geotiff into numpy arrays

In [ ]:
import rasterio

## Multiband

In [ ]:
dataset = rasterio.open("../data/raster/Mumbai_2016.tif")
data=dataset.read(1)

## Single band

In [ ]:
dataset = rasterio.open("../data/raster/LC81480472016284LGN00_B6.tif")
data=dataset.read(1)

In [ ]:
data.shape

## Subset a satellite imagery

In [ ]:
import fiona
import rasterio
import rasterio.tools.mask
import glob
import os
import ntpath


with fiona.open("Mumbai_extent_32643.shp", "r") as shapefile:
    features = [feature["geometry"] for feature in shapefile]


lcfolders='mumbai/LC81480472016284LGN00'
    
def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

tifiles=glob.glob(lcfolders+'/*.TIF')
for tifile in tifiles:
    shpfiles2=(os.path.splitext(tifile)[0])
    shpfile1=path_leaf(shpfiles2)
    with rasterio.open(tifile) as src:
        out_image, out_transform = rasterio.tools.mask.mask(src, features,crop=True)
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff","height": out_image.shape[1],"width": out_image.shape[2],"transform": out_transform})
        with rasterio.open('/home/sunbird/treecover/Msubset/'+shpfile1+".tif", "w", **out_meta) as dest:
            dest.write(out_image)